Feature Extraction

In [1]:
from urllib.parse import urlparse,urlencode
import ipaddress
import re
     

3.1 IP Address in the URL
Checks for the presence of IP address in the URL. URLs may have IP address instead of domain name. If an IP address is used as an alternative of the domain name in the URL, we can be sure that someone is trying to steal personal information with this URL.

If the domain part of URL has IP address, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

re.search() function is used to search for specific patters in a url.

In [3]:
def has_ip_address(url):
    # Regular expression pattern to match IPv4 addresses
    ipv4_pattern = r'\b(?:\d{1,3}\.){3}\d{1,3}\b'
    
    # Regular expression pattern to match IPv6 addresses
    ipv6_pattern = r'\b(?:[A-F0-9]{1,4}:){7}[A-F0-9]{1,4}\b'
    
    # Search for IPv4 and IPv6 patterns in the URL
    has_ipv4 = re.search(ipv4_pattern, url) is not None
    has_ipv6 = re.search(ipv6_pattern, url) is not None
    
    # Return True if either IPv4 or IPv6 address is found in the URL
    if (has_ipv4 | has_ipv6):
        ip = 1 #has ip address
    else:
        ip = 0 #doesn't have ip address
    
    return ip

3.2 Checking the length of the URL

if length of URL >= 54 then assigning it a value of 1 (phishing) else 0 (benign)

In [4]:
def length(url):
    if len(url) < 54:
        len = 0
    else:
        len = 1
    return len

3.3 If there is a @ in a url, everything after the @ is ignored by the browser
We assign at a value of 1 (phishing) if the url has @ else 0 (benign)

In [6]:
def checkAtSign(url):
    if "@" in url:
        at = 1
    else:
        at = 0
    return at

3.4 Domain Age: Phishing websites often have recently registered domains, while legitimate websites tend to have older domains. You can determine the age of the domain using WHOIS data and include it as a feature.

In [20]:
!pip install python-whois
from datetime import datetime 
import whois


In [19]:
def calc_domain_age(url, threshold=30):
    try:
        domain_information = whois.whois(domain)
        creation_date = domain_information.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
            
        if creation_date:
            current_date = datetime.now()
            domain_age = (current_date - creation_date).days
        else:
            domain_age = None

        if domain_age and domain_age <= threshold:
            recent_domain = 1
        else:
            recent_domain = 0
        return recent_domain
        
    except:
        return 0

3.5 URL shortening
URL shortening is a method on the “World Wide Web” in which a URL may be made considerably smaller in length and still lead to the required webpage. 

If the URL is using Shortening Services, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [23]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [24]:
def check_shortening(url):
    match = re.search(shortening_services, url)
    if match:
        return 1
    else:
        return 0

In [25]:
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
from datetime import datetime

3.6 Web traffic
Phishing websites usually live for a short period of time; they may not be recognised by the Alexa database. 

For our model, if the domain has no traffic or is not recognized by the Alexa database, it is classified as “Phishing”.


In [ ]:
def web_traffic(url):
  try:
    #Filling the whitespaces in the URL if any
    url = urllib.parse.quote(url)
    rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
    rank = int(rank)
  except TypeError:
        return 1
  if rank <100000:
    return 1
  else:
    return 0